## Imports

In [1]:
import sys 
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline  

from sklearn import svm, linear_model
from sklearn.linear_model import SGDClassifier
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.externals import joblib

import torch
import torchvision 
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset, DataLoader

sys.path.insert(0, '../../../Utils/')
sys.path.insert(0, '../../../')
import cyphercat as cc

import models
from train import *
from metrics import * 
from SVC_Utils import *

#audio
import librosa as libr

print("Python: %s" % sys.version)
print("Pytorch: %s" % torch.__version__)

# determine device to run network on (runs on gpu if available)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


Python: 3.6.5 (default, Jul  6 2018, 19:12:46) 
[GCC 5.4.0 20160609]
Pytorch: 0.4.0


## NN Hyperparameters

In [2]:
batch_size = 128
lr = 0.001
k = 3

pretrained = False #run this with networks that have already been trained

transform_type = 'SFTF' #either STFT or MFCC  

data = 'Libri' #'Libri' or 'VOiCES'

In [3]:
### Speech preprocessing

class tensorToMFCC:
    def __call__(self, y):
#         y = y.numpy()
        dims = y.shape
        y = libr.feature.melspectrogram(np.reshape(y, (dims[1],)), 16000, n_mels=number_of_mels,
                               fmax=8000)
        y = libr.feature.mfcc(S = libr.power_to_db(y))
        y = torch.from_numpy(y)                           
        return y.float()

class STFT:
    def __call__(self,y):
        dims = y.shape
        y = np.abs(libr.core.stft(np.reshape(y, (dims[1],))))
        y = torch.from_numpy(y).permute(1,0)
        return y.float()

if transform_type == 'SFTF':
    target_net_type = cc.ft_cnn_classifer
    shadow_net_type = cc.ft_cnn_classifer
    in_size = 94# 20 forMFCC,  94 for STFT
    transform  = STFT() ## STFT or MFCC
elif transform_type == 'MFCC':
    transform  = tensorToMFCC()
    target_net_type = cc.MFCC_cnn_classifier
    shadow_net_type = cc.MFCC_cnn_classifier
    in_size = 20

## Audio hyperparameters

In [4]:
n_seconds = 3
n_epochs = 15
sampling_rate = 16000
number_of_mels =128
lr = 0.001

# attacking means data for a target & shadow network.
# This will also split "out data" from totally different speakers -- data none of the 
# other networks have seen, for training & testing the attack network. This will be
# an equivalent amount of data to the train split as defined about

%load_ext autoreload
%autoreload 2
sys.path.insert(0, './../../../Utils')

## Load audio data: VOiCES or LibriSpeech, & Split into valid sequences

In [5]:
print('Loading splits')

if data == 'Libri':
    dfs = cc.Libri_preload_and_split()
    print('Initializing dataset')
    valid_sequence_train_target = cc.LibriSpeechDataset(df=dfs[0], transform = transform)
    valid_sequence_test_target = cc.LibriSpeechDataset(df=dfs[1], transform = transform)
    valid_sequence_train_shadow = cc.LibriSpeechDataset(df=dfs[2], transform = transform)
    valid_sequence_test_shadow = cc.LibriSpeechDataset(df=dfs[3], transform = transform)
    valid_sequence_test = cc.LibriSpeechDataset(df=dfs[4], transform = transform)

    print('Succesfully loaded libri-speech')
elif data == 'VOiCES':
    print('Loading splits')
    dfs = cc.Voices_preload_and_split()
    print('Initializing dataset')
    valid_sequence_train_target = cc.Voices_dataset(df=dfs[0], transform = transform)
    valid_sequence_test_target = cc.Voices_dataset(df=dfs[1], transform = transform)
    valid_sequence_train_shadow = cc.Voices_dataset(df=dfs[2], transform = transform)
    valid_sequence_test_shadow = cc.Voices_dataset(df=dfs[3], transform = transform)
    valid_sequence_test = cc.Voices_dataset(df=dfs[4], transform = transform)

Loading splits
Initialising LibriSpeechDataset with minimum length = 3s and subset = train-clean-100
Finished indexing data. 27949 usable files found.
Found default splits, loading dataframe
		 ---- Split 0 ---- 
	Unique speakers 	 Samples
Male:		 63		 5539
Female:		 62		 5573
Total:		 125		 11112
		 ---- Split 1 ---- 
	Unique speakers 	 Samples
Male:		 63		 1414
Female:		 62		 1427
Total:		 125		 2841
		 ---- Split 2 ---- 
	Unique speakers 	 Samples
Male:		 63		 3471
Female:		 63		 3490
Total:		 126		 6961
		 ---- Split 3 ---- 
	Unique speakers 	 Samples
Male:		 63		 3510
Female:		 63		 3525
Total:		 126		 7035
		 ---- Split 4 ---- 
	Unique speakers 	 Samples
Male:		 16		 1414
Female:		 16		 1427
Total:		 32		 2841
Finished splitting data.
Initializing dataset
Succesfully loaded libri-speech


In [6]:
batch_size = 32


# Loaders for data for target model & shadow model 
target_train_loader = DataLoader(valid_sequence_train_target,
                      batch_size=batch_size,
                      shuffle=True,
                      num_workers=8,
                    drop_last = True
                     # pin_memory=True # CUDA only
                     )


target_test_loader = DataLoader(valid_sequence_test_target,
                      batch_size=batch_size,
                      shuffle=True,
                      num_workers=8
                     # pin_memory=True # CUDA only
                     )

shadow_train_loader = DataLoader(valid_sequence_train_shadow,
                      batch_size=batch_size,
                      shuffle=True,
                      num_workers=8,
                    drop_last = True
                     # pin_memory=True # CUDA only
                     )


shadow_test_loader = DataLoader(valid_sequence_test_shadow,
                      batch_size=batch_size,
                      shuffle=True,
                      num_workers=8
                     # pin_memory=True # CUDA only
                     )


test_loader = DataLoader(valid_sequence_test,
                      batch_size=batch_size,
                      shuffle=True,
                      num_workers=8
                     # pin_memory=True # CUDA only
                     )


## Set up model tracking

In [16]:
#table with summary

# To do: extract accuracy from train/eval funcs and automatically add to table
summary_file = 'summary.pkl'
columns = ['Transform','Training epochs', '# speakers','Train accuracy', 'Test accuracy', 'Attack type', 'Precision','Recall']

try:
    df = pd.read_pickle(summary_file)

except:
    df = pd.DataFrame(columns = columns)
    
df_idx = len(df)

#set a bunch of known values
df.at[df_idx,'Transform'] =transform_type
df.at[df_idx,'Training epochs'] = n_epochs
df.at[df_idx,'Attack type'] = 1

# Initialize/Train Targets
The model being attacked; if network, architecture can differ from that of shadow network.

In [17]:
#Initialize NN

#in_size defined above
n_hidden = 512
n_classes = valid_sequence_test_target.num_speakers
print(n_classes,' speakers')
df.at[df_idx,'# speakers']=n_classes


target_net = target_net_type(n_classes).to(device)
target_net.apply(models.weights_init)

target_loss = nn.CrossEntropyLoss()
target_optim = optim.Adam(target_net.parameters(), lr=lr)

125  speakers


In [9]:
#file name for this set of hyperparameters
fn = 'model_weights/CNN_voice_classifier'+data+'_target_'+transform_type+str(n_epochs-1)+'.pth'

#Train NN
if not pretrained:
    train_accuracy, test_accuracy = cc.train(target_net, target_train_loader, target_test_loader, target_optim, target_loss, n_epochs, verbose = False) 
    
    cc.save_checkpoint(model = target_net, optimizer = target_optim,
                           epoch = n_epochs-1, data_descriptor = data, 
                           accuracy = [train_accuracy, test_accuracy],
                           filename = fn)
    
else:
    cc.load_checkpoint(model = target_net, optimizer = target_optim, checkpoint = fn)


[0/15]
Training:

Accuracy = 67.27 %%


Test:

Accuracy = 56.35 %%


[1/15]
Training:

Accuracy = 83.80 %%


Test:

Accuracy = 74.52 %%


[2/15]
Training:

Accuracy = 90.62 %%


Test:

Accuracy = 82.68 %%


[3/15]
Training:

Accuracy = 92.23 %%


Test:

Accuracy = 83.49 %%


[4/15]
Training:

Accuracy = 93.88 %%


Test:

Accuracy = 86.10 %%


[5/15]
Training:

Accuracy = 94.19 %%


Test:

Accuracy = 87.26 %%


[6/15]
Training:

Accuracy = 94.65 %%


Test:

Accuracy = 86.41 %%


[7/15]
Training:

Accuracy = 97.04 %%


Test:

Accuracy = 89.55 %%


[8/15]
Training:

Accuracy = 93.88 %%


Test:

Accuracy = 85.81 %%


[9/15]
Training:

Accuracy = 97.02 %%


Test:

Accuracy = 91.02 %%


[10/15]
Training:

Accuracy = 96.80 %%


Test:

Accuracy = 89.97 %%


[11/15]
Training:

Accuracy = 97.29 %%


Test:

Accuracy = 92.43 %%


[12/15]
Training:

Accuracy = 97.43 %%


Test:

Accuracy = 92.33 %%


[13/15]
Training:

Accuracy = 97.88 %%


Test:

Accuracy = 92.15 %%


[14/15]
Training:

Accuracy = 

In [18]:
df.at[df_idx,'Train accuracy'] =round(train_accuracy,4)
df.at[df_idx,'Test accuracy'] = round(test_accuracy,4)

# Initialize/Train Shadow Model
Shadow model mimics the target network, emulating the target model's differences in prediction probabilities for samples in and out of its dataset. For this attack, only one shadow model is used. 

In [20]:
#Initialize models

n_classes = valid_sequence_test_shadow.num_speakers
print('n shadow speakers',n_classes)

#NN
shadow_net = shadow_net_type(n_classes).to(device)
shadow_net.apply(models.weights_init)

shadow_loss = nn.CrossEntropyLoss()
shadow_optim = optim.Adam(shadow_net.parameters(), lr=lr)
shadow_epochs = 15 #n_epochs

n shadow speakers 126


# Initialize Attack Model
A binary classifier to determine membership. 

In [21]:
# Attack the network: 

attack_net_nn = models.mlleaks_mlp(n_in=k).to(device)
attack_loss = nn.BCEWithLogitsLoss()
attack_optim_nn= optim.Adam(attack_net_nn.parameters(), lr=lr)
n_epochs_attack = 15

df_pr = cc.ml_leaks1(target=target_net, shadow_model = shadow_net, attacker_model = attack_net_nn,
            target_in_loader = test_loader, target_out_loader = target_test_loader,
            shadow_train_loader = shadow_train_loader, shadow_out_loader=shadow_test_loader,
            shadow_optim = shadow_optim, attack_optim = attack_optim_nn, 
            shadow_criterion = shadow_loss, attack_criterion = attack_loss, 
            shadow_epochs = shadow_epochs, attack_epochs = n_epochs_attack, retrain = True)

---- Training shadow network ----
[0/15]
Training:

Accuracy = 27.82 %%


Test:

Accuracy = 21.49 %%


[1/15]
Training:

Accuracy = 64.04 %%


Test:

Accuracy = 48.77 %%


[2/15]
Training:

Accuracy = 77.28 %%


Test:

Accuracy = 60.17 %%


[3/15]
Training:

Accuracy = 86.39 %%


Test:

Accuracy = 67.39 %%


[4/15]
Training:

Accuracy = 88.72 %%


Test:

Accuracy = 71.13 %%


[5/15]
Training:

Accuracy = 90.14 %%


Test:

Accuracy = 72.08 %%


[6/15]
Training:

Accuracy = 89.80 %%


Test:

Accuracy = 72.24 %%


[7/15]
Training:

Accuracy = 93.39 %%


Test:

Accuracy = 74.95 %%


[8/15]
Training:

Accuracy = 94.04 %%


Test:

Accuracy = 75.78 %%


[9/15]
Training:

Accuracy = 95.09 %%


Test:

Accuracy = 78.85 %%


[10/15]
Training:

Accuracy = 95.38 %%


Test:

Accuracy = 78.08 %%


[11/15]
Training:

Accuracy = 95.26 %%


Test:

Accuracy = 77.64 %%


[12/15]
Training:

Accuracy = 96.83 %%


Test:

Accuracy = 80.55 %%


[13/15]
Training:

Accuracy = 95.39 %%


Test:

Accuracy = 77.78 %

[0/15][159/217] loss = 0.69, accuracy = 55.02
[0/15][160/217] loss = 0.69, accuracy = 55.02
[0/15][161/217] loss = 0.69, accuracy = 55.03
[0/15][162/217] loss = 0.69, accuracy = 55.10
[0/15][163/217] loss = 0.69, accuracy = 55.14
[0/15][164/217] loss = 0.69, accuracy = 55.18
[0/15][165/217] loss = 0.69, accuracy = 55.21
[0/15][166/217] loss = 0.69, accuracy = 55.30
[0/15][167/217] loss = 0.69, accuracy = 55.31
[0/15][168/217] loss = 0.69, accuracy = 55.32
[0/15][169/217] loss = 0.69, accuracy = 55.32
[0/15][170/217] loss = 0.69, accuracy = 55.32
[0/15][171/217] loss = 0.69, accuracy = 55.29
[0/15][172/217] loss = 0.69, accuracy = 55.28
[0/15][173/217] loss = 0.69, accuracy = 55.33
[0/15][174/217] loss = 0.69, accuracy = 55.37
[0/15][175/217] loss = 0.69, accuracy = 55.42
[0/15][176/217] loss = 0.69, accuracy = 55.41
[0/15][177/217] loss = 0.69, accuracy = 55.42
[0/15][178/217] loss = 0.69, accuracy = 55.44
[0/15][179/217] loss = 0.69, accuracy = 55.45
[0/15][180/217] loss = 0.69, accur

[1/15][126/217] loss = 0.69, accuracy = 60.20
[1/15][127/217] loss = 0.68, accuracy = 60.21
[1/15][128/217] loss = 0.68, accuracy = 60.16
[1/15][129/217] loss = 0.70, accuracy = 60.11
[1/15][130/217] loss = 0.67, accuracy = 60.15
[1/15][131/217] loss = 0.68, accuracy = 60.16
[1/15][132/217] loss = 0.68, accuracy = 60.17
[1/15][133/217] loss = 0.68, accuracy = 60.18
[1/15][134/217] loss = 0.68, accuracy = 60.13
[1/15][135/217] loss = 0.69, accuracy = 60.14
[1/15][136/217] loss = 0.68, accuracy = 60.10
[1/15][137/217] loss = 0.68, accuracy = 60.11
[1/15][138/217] loss = 0.68, accuracy = 60.08
[1/15][139/217] loss = 0.67, accuracy = 60.11
[1/15][140/217] loss = 0.69, accuracy = 60.07
[1/15][141/217] loss = 0.69, accuracy = 60.02
[1/15][142/217] loss = 0.68, accuracy = 60.02
[1/15][143/217] loss = 0.68, accuracy = 60.00
[1/15][144/217] loss = 0.69, accuracy = 59.97
[1/15][145/217] loss = 0.68, accuracy = 59.95
[1/15][146/217] loss = 0.69, accuracy = 59.92
[1/15][147/217] loss = 0.67, accur

[2/15][94/217] loss = 0.68, accuracy = 59.59
[2/15][95/217] loss = 0.68, accuracy = 59.60
[2/15][96/217] loss = 0.66, accuracy = 59.70
[2/15][97/217] loss = 0.68, accuracy = 59.65
[2/15][98/217] loss = 0.69, accuracy = 59.60
[2/15][99/217] loss = 0.65, accuracy = 59.70
[2/15][100/217] loss = 0.67, accuracy = 59.75
[2/15][101/217] loss = 0.66, accuracy = 59.77
[2/15][102/217] loss = 0.68, accuracy = 59.74
[2/15][103/217] loss = 0.67, accuracy = 59.72
[2/15][104/217] loss = 0.68, accuracy = 59.72
[2/15][105/217] loss = 0.68, accuracy = 59.70
[2/15][106/217] loss = 0.66, accuracy = 59.74
[2/15][107/217] loss = 0.65, accuracy = 59.82
[2/15][108/217] loss = 0.66, accuracy = 59.91
[2/15][109/217] loss = 0.67, accuracy = 59.96
[2/15][110/217] loss = 0.67, accuracy = 59.97
[2/15][111/217] loss = 0.65, accuracy = 60.09
[2/15][112/217] loss = 0.67, accuracy = 60.11
[2/15][113/217] loss = 0.68, accuracy = 60.06
[2/15][114/217] loss = 0.65, accuracy = 60.08
[2/15][115/217] loss = 0.68, accuracy = 

Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7fcd62e3a048>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 349, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 328, in _shutdown_workers
    self.worker_result_queue.get()
  File "/usr/local/lib/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
  File "/usr/local/lib/python3.6/site-packages/torch/multiprocessing/reductions.py", line 70, in rebuild_storage_fd
    fd = df.detach()
  File "/usr/local/lib/python3.6/multiprocessing/resource_sharer.py", line 58, in detach
    return reduction.recv_handle(conn)
  File "/usr/local/lib/python3.6/multiprocessing/reduction.py", line 182, in recv_handle
    return recvfds(s, 1)[0]
  File "/usr/local/lib/python3.6/multiprocessing/reducti

...

[12/15][216/217] loss = 0.62, accuracy = 60.25


Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7fcd5fdcb748>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 349, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 328, in _shutdown_workers
    self.worker_result_queue.get()
  File "/usr/local/lib/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
  File "/usr/local/lib/python3.6/site-packages/torch/multiprocessing/reductions.py", line 70, in rebuild_storage_fd
    fd = df.detach()
  File "/usr/local/lib/python3.6/multiprocessing/resource_sharer.py", line 58, in detach
    return reduction.recv_handle(conn)
  File "/usr/local/lib/python3.6/multiprocessing/reduction.py", line 182, in recv_handle
    return recvfds(s, 1)[0]
  File "/usr/local/lib/python3.6/multiprocessing/reducti

[13/15][0/217] loss = 0.63, accuracy = 64.06
[13/15][1/217] loss = 0.66, accuracy = 64.06
[13/15][2/217] loss = 0.66, accuracy = 59.90
[13/15][3/217] loss = 0.66, accuracy = 59.77
[13/15][4/217] loss = 0.63, accuracy = 60.31
[13/15][5/217] loss = 0.61, accuracy = 61.20
[13/15][6/217] loss = 0.64, accuracy = 60.94
[13/15][7/217] loss = 0.68, accuracy = 60.55
[13/15][8/217] loss = 0.64, accuracy = 61.11
[13/15][9/217] loss = 0.72, accuracy = 60.00
[13/15][10/217] loss = 0.67, accuracy = 59.52
[13/15][11/217] loss = 0.66, accuracy = 59.38
[13/15][12/217] loss = 0.73, accuracy = 58.53
[13/15][13/217] loss = 0.67, accuracy = 58.93
[13/15][14/217] loss = 0.67, accuracy = 58.85
[13/15][15/217] loss = 0.68, accuracy = 58.89
[13/15][16/217] loss = 0.63, accuracy = 59.19
[13/15][17/217] loss = 0.66, accuracy = 59.29
[13/15][18/217] loss = 0.71, accuracy = 59.05
[13/15][19/217] loss = 0.66, accuracy = 59.06
[13/15][20/217] loss = 0.67, accuracy = 59.23
[13/15][21/217] loss = 0.68, accuracy = 59.0

[13/15][178/217] loss = 0.68, accuracy = 59.72
[13/15][179/217] loss = 0.67, accuracy = 59.73
[13/15][180/217] loss = 0.70, accuracy = 59.69
[13/15][181/217] loss = 0.68, accuracy = 59.72
[13/15][182/217] loss = 0.63, accuracy = 59.75
[13/15][183/217] loss = 0.74, accuracy = 59.73
[13/15][184/217] loss = 0.64, accuracy = 59.75
[13/15][185/217] loss = 0.64, accuracy = 59.74
[13/15][186/217] loss = 0.69, accuracy = 59.71
[13/15][187/217] loss = 0.66, accuracy = 59.70
[13/15][188/217] loss = 0.71, accuracy = 59.68
[13/15][189/217] loss = 0.69, accuracy = 59.64
[13/15][190/217] loss = 0.71, accuracy = 59.60
[13/15][191/217] loss = 0.61, accuracy = 59.65
[13/15][192/217] loss = 0.62, accuracy = 59.67
[13/15][193/217] loss = 0.66, accuracy = 59.67
[13/15][194/217] loss = 0.66, accuracy = 59.69
[13/15][195/217] loss = 0.73, accuracy = 59.64
[13/15][196/217] loss = 0.69, accuracy = 59.61
[13/15][197/217] loss = 0.68, accuracy = 59.62
[13/15][198/217] loss = 0.62, accuracy = 59.67
[13/15][199/2

[14/15][138/217] loss = 0.64, accuracy = 59.88
[14/15][139/217] loss = 0.71, accuracy = 59.83
[14/15][140/217] loss = 0.69, accuracy = 59.82
[14/15][141/217] loss = 0.67, accuracy = 59.78
[14/15][142/217] loss = 0.62, accuracy = 59.82
[14/15][143/217] loss = 0.67, accuracy = 59.81
[14/15][144/217] loss = 0.66, accuracy = 59.82
[14/15][145/217] loss = 0.69, accuracy = 59.79
[14/15][146/217] loss = 0.70, accuracy = 59.80
[14/15][147/217] loss = 0.66, accuracy = 59.80
[14/15][148/217] loss = 0.75, accuracy = 59.72
[14/15][149/217] loss = 0.69, accuracy = 59.72
[14/15][150/217] loss = 0.68, accuracy = 59.68
[14/15][151/217] loss = 0.62, accuracy = 59.68
[14/15][152/217] loss = 0.66, accuracy = 59.67
[14/15][153/217] loss = 0.71, accuracy = 59.59
[14/15][154/217] loss = 0.66, accuracy = 59.62
[14/15][155/217] loss = 0.64, accuracy = 59.65
[14/15][156/217] loss = 0.66, accuracy = 59.64
[14/15][157/217] loss = 0.65, accuracy = 59.67
[14/15][158/217] loss = 0.65, accuracy = 59.67
[14/15][159/2

In [22]:
# Ascertain best results

df.at[df_idx,'Precision'] = round(df_pr[df_pr['Accuracy']==df_pr['Accuracy'].max()].Precision.values[0],4)
df.at[df_idx,'Recall'] = round(df_pr[df_pr['Accuracy']==df_pr['Accuracy'].max()].Recall.values[0],4)


In [31]:
df

,Transform,Training epochs,# speakers,Train accuracy,Test accuracy,Attack type,Precision,Recall
0,SFTF,15.0,125.0,97.7035,92.4322,1.0,0.5202,0.8698


In [13]:
df.to_pickle(summary_file)

# Evaluate Attack Nets
How well the trained attack models classify a sample as in or out of a target model's training dataset, and how performance is affected by target hyperparameters and which models attack which targets.

In [37]:
def make_float(df,col):
    df[col]= df[col].astype(float)
    return df

for col in ['# speakers','Training epochs','Train accuracy',
            'Test accuracy','Attack type']:
    df = make_float(df,col)

#style table
import seaborn as sns

cg = sns.light_palette("green", as_cmap=True)
cm = sns.light_palette("magenta", as_cmap=True)
bl = sns.light_palette("blue", as_cmap=True)
orr = sns.light_palette("orange", as_cmap=True)
gr = sns.light_palette("gray", as_cmap=True)

# df.style.bar(subset=['Train accuracy', 'Test accuracy'], align='mid', color=['#d65f5f', '#5fba7d'])
s = df.style.\
    background_gradient(cmap=cg,subset=['Train accuracy', 'Test accuracy']).\
    background_gradient(cmap=bl,subset=['Precision', 'Recall']).\
    background_gradient(cmap=orr,subset=['Training epochs']).\
    background_gradient(cmap=gr,subset=['Attack type']).\
    background_gradient(cmap=cm,subset=['# speakers']).\
    format({"Train accuracy": "{:.2%}","Test accuracy": "{:.2%}"}).\
    hide_index().\
    set_properties(**{'font-size': "16pt",'column-size':"24pt",'width': '100px'})

s

TypeError: ("Cannot cast array data from dtype('O') to dtype('int64') according to the rule 'safe'", 'occurred at index Precision')

In [15]:
#old, for reference for now:

# do this for 10 & 100 speakers
# .2 S & 3 S
#sufficient training and over-training

#manual data: 

#Attack 1:
df.loc[len(df)] = ['MFCC',25,69.0,.9994,.9632,1,0.89,0.90] 
df.loc[len(df)] = ['MFCC',25,255.0,.9961,.9443,1,0.88,0.91] 
df.loc[len(df)] = ['STFT',25,69.0,0.9989,0.9451,1,0.89,0.92] 
df.loc[len(df)] = ['STFT',25,255.0,0.9958,0.9181,1,0.85,0.86] 

#Attack 3 w/max data: 
df.loc[len(df)] = ['STFT',25,139.0,.9985,.9073,3,.81,.90] 
df.loc[len(df)] = ['STFT',50,511.0,.9942,.9057,3,.84,.87] 
df.loc[len(df)] = ['MFCC',25,139.0,.9969,.9136,3,.82,.92] 
df.loc[len(df)] = ['MFCC',25,511.0,.9960,.9321,3,0.83,0.93]


#Attack 3 on Attack1 models:
df.loc[len(df)] = ['MFCC',25,69.0,.9994,.9632,3,0.84,0.95] 
df.loc[len(df)] = ['MFCC',25,255.0,.9961,.9443,3,0.84,0.94] 
df.loc[len(df)] = ['STFT',25,69.0,0.9989,0.9451,3,0.81,0.97] 
df.loc[len(df)] = ['STFT',25,255.0,0.9958,0.9181,3,0.81,0.90] 

df['# speakers'] =df['# speakers'].astype(float)
df['Training epochs'] =df['Training epochs'].astype(float)
df['Attack type'] =df['Attack type'].astype(float)

#style table
import seaborn as sns

cg = sns.light_palette("green", as_cmap=True)
cm = sns.light_palette("magenta", as_cmap=True)
bl = sns.light_palette("blue", as_cmap=True)
orr = sns.light_palette("orange", as_cmap=True)
gr = sns.light_palette("gray", as_cmap=True)

# df.style.bar(subset=['Train accuracy', 'Test accuracy'], align='mid', color=['#d65f5f', '#5fba7d'])
s = df.style.\
    background_gradient(cmap=cg,subset=['Train accuracy', 'Test accuracy']).\
    background_gradient(cmap=bl,subset=['Precision', 'Recall']).\
    background_gradient(cmap=orr,subset=['Training epochs']).\
    background_gradient(cmap=gr,subset=['Attack type']).\
    background_gradient(cmap=cm,subset=['# speakers']).\
    format({"Train accuracy": "{:.2%}","Test accuracy": "{:.2%}"}).\
    hide_index().\
    set_properties(**{'font-size': "16pt",'column-size':"24pt",'width': '100px'})

s

TypeError: ("Cannot cast array data from dtype('O') to dtype('int64') according to the rule 'safe'", 'occurred at index Train accuracy')